## Import Packages

In [1]:
import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random
import json

from PIL import Image, ImageOps, ImageEnhance
import numbers

import torchattacks
from torchattacks import CW

import matplotlib.pyplot as plt
%matplotlib inline

## Normalize Dataset

In [2]:
#Creating dataset using torch dataloaders
batch_size_train = 64
batch_size_test = 1000
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))]) # Normalizing dataset

# Training dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=transform),
  batch_size=batch_size_train, shuffle=True)

# Test dataset
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=transform),
  batch_size=batch_size_test, shuffle=True)

# Initialize GPU
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


## Initial Model Structure

In [3]:
# Initial model structure
class HNet(nn.Module):    
    def __init__(self):
        super(HNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x     

## SubModel Structure

In [4]:
# Submodel Structure for training residual
class NHNet(nn.Module):    
    def __init__(self):
        super(NHNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x   

## Initialize Data Structures

In [5]:
# Initialize some data structures to store useful data
train_losses = []
train_counter = []
test_losses = []

## Initialize Parameters, optimizer and loss function

In [6]:
# Create the initial model
initial_model = HNet()

#Create the optimizer for the initial model
optimizer = optim.Adam(initial_model.parameters(), lr=0.003)

# Create Loss function for the intial model
criterion = nn.CrossEntropyLoss()

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Change model into cuda mode
if torch.cuda.is_available():
    initial_model = initial_model.cuda()
    criterion = criterion.cuda()

## Training Function for Initial Model

In [7]:
# Initial Model Training Function
def train(epoch):
    initial_model.train()
#     exp_lr_scheduler.step()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = initial_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        # torch.save(initial_model.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/model.pth')
        # torch.save(optimizer.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/optimizer.pth')
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

## Evaluation Function for Intial Model

In [8]:
#Initial Model Evaluating Function
def evaluate(data_loader):
    initial_model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
        
            output = initial_model(data)
        
            loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    test_losses.append(loss)    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

## Start Training the Initial Model

In [9]:
# Start Training
n_epochs = 65

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6400/60000 (11%)]	Loss: 1.661364
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.601474
Train Epoch: 0 [19200/60000 (32%)]	Loss: 1.621534
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.655728
Train Epoch: 0 [32000/60000 (53%)]	Loss: 1.572359
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.599732
Train Epoch: 0 [44800/60000 (75%)]	Loss: 1.547913
Train Epoch: 0 [51200/60000 (85%)]	Loss: 1.559445
Train Epoch: 0 [57600/60000 (96%)]	Loss: 1.511098


C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':



Average loss: 1.5579, Accuracy: 54310/60000 (90.517%)

Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.528747
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.615363
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.541097
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.558620
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.527702
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.518819
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.530707
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.560704
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.515562

Average loss: 1.5430, Accuracy: 55119/60000 (91.865%)

Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.537167
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.549802
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.491152
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.537123
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.493295
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.560355
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.603831
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.559644
Train Epoch: 2 [57600/60000 (96%)]	Loss:

Train Epoch: 17 [6400/60000 (11%)]	Loss: 1.555029
Train Epoch: 17 [12800/60000 (21%)]	Loss: 1.539479
Train Epoch: 17 [19200/60000 (32%)]	Loss: 1.539276
Train Epoch: 17 [25600/60000 (43%)]	Loss: 1.508026
Train Epoch: 17 [32000/60000 (53%)]	Loss: 1.554901
Train Epoch: 17 [38400/60000 (64%)]	Loss: 1.523724
Train Epoch: 17 [44800/60000 (75%)]	Loss: 1.492603
Train Epoch: 17 [51200/60000 (85%)]	Loss: 1.543978
Train Epoch: 17 [57600/60000 (96%)]	Loss: 1.508083

Average loss: 1.5426, Accuracy: 55116/60000 (91.860%)

Train Epoch: 18 [6400/60000 (11%)]	Loss: 1.523651
Train Epoch: 18 [12800/60000 (21%)]	Loss: 1.511070
Train Epoch: 18 [19200/60000 (32%)]	Loss: 1.476776
Train Epoch: 18 [25600/60000 (43%)]	Loss: 1.523651
Train Epoch: 18 [32000/60000 (53%)]	Loss: 1.585746
Train Epoch: 18 [38400/60000 (64%)]	Loss: 1.508026
Train Epoch: 18 [44800/60000 (75%)]	Loss: 1.554097
Train Epoch: 18 [51200/60000 (85%)]	Loss: 1.519577
Train Epoch: 18 [57600/60000 (96%)]	Loss: 1.569870

Average loss: 1.5138, Accur

Train Epoch: 33 [6400/60000 (11%)]	Loss: 1.492422
Train Epoch: 33 [12800/60000 (21%)]	Loss: 1.617339
Train Epoch: 33 [19200/60000 (32%)]	Loss: 1.539276
Train Epoch: 33 [25600/60000 (43%)]	Loss: 1.508026
Train Epoch: 33 [32000/60000 (53%)]	Loss: 1.523653
Train Epoch: 33 [38400/60000 (64%)]	Loss: 1.586114
Train Epoch: 33 [44800/60000 (75%)]	Loss: 1.508026
Train Epoch: 33 [51200/60000 (85%)]	Loss: 1.523651
Train Epoch: 33 [57600/60000 (96%)]	Loss: 1.508026

Average loss: 1.5344, Accuracy: 55598/60000 (92.663%)

Train Epoch: 34 [6400/60000 (11%)]	Loss: 1.570526
Train Epoch: 34 [12800/60000 (21%)]	Loss: 1.492401
Train Epoch: 34 [19200/60000 (32%)]	Loss: 1.508026
Train Epoch: 34 [25600/60000 (43%)]	Loss: 1.492404
Train Epoch: 34 [32000/60000 (53%)]	Loss: 1.586151
Train Epoch: 34 [38400/60000 (64%)]	Loss: 1.586151
Train Epoch: 34 [44800/60000 (75%)]	Loss: 1.648649
Train Epoch: 34 [51200/60000 (85%)]	Loss: 1.554901
Train Epoch: 34 [57600/60000 (96%)]	Loss: 1.601776

Average loss: 1.5411, Accur

Train Epoch: 49 [6400/60000 (11%)]	Loss: 1.547493
Train Epoch: 49 [12800/60000 (21%)]	Loss: 1.554901
Train Epoch: 49 [19200/60000 (32%)]	Loss: 1.508026
Train Epoch: 49 [25600/60000 (43%)]	Loss: 1.570526
Train Epoch: 49 [32000/60000 (53%)]	Loss: 1.570526
Train Epoch: 49 [38400/60000 (64%)]	Loss: 1.570526
Train Epoch: 49 [44800/60000 (75%)]	Loss: 1.508026
Train Epoch: 49 [51200/60000 (85%)]	Loss: 1.523651
Train Epoch: 49 [57600/60000 (96%)]	Loss: 1.585912

Average loss: 1.5399, Accuracy: 55272/60000 (92.120%)

Train Epoch: 50 [6400/60000 (11%)]	Loss: 1.633026
Train Epoch: 50 [12800/60000 (21%)]	Loss: 1.539276
Train Epoch: 50 [19200/60000 (32%)]	Loss: 1.601776
Train Epoch: 50 [25600/60000 (43%)]	Loss: 1.523676
Train Epoch: 50 [32000/60000 (53%)]	Loss: 1.554901
Train Epoch: 50 [38400/60000 (64%)]	Loss: 1.508026
Train Epoch: 50 [44800/60000 (75%)]	Loss: 1.539276
Train Epoch: 50 [51200/60000 (85%)]	Loss: 1.539300
Train Epoch: 50 [57600/60000 (96%)]	Loss: 1.508026

Average loss: 1.5338, Accur

## Save or Load the Initial Model

In [10]:
# Save this model so that I dont have to train again in the future
torch.save(initial_model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')

In [7]:
# Load the model from local file
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

HNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

## Calculatee mse residual and normalize the result to values between 0 ~ 1

In [8]:
# Function used to calculate mse residual based on the wiki
def mseresidual(y, F):
    residual = y - F
    absolute = torch.abs(residual)
    residual = residual / torch.max(absolute)
    return residual

## Actual Code for gradient Boosting

In [28]:
#Main Function used to train and find optimal gamma for submodels
#input: intial model that's already trained
#M is number of submodels needed to be trained
def GradientBoosting(initial_model, M):
    gamma_exp = torch.ones([M], dtype = torch.float64) # used to hold the final optimized gamma
    models = [] # used to hold all the models
    residual_list = [] # used to hold the residual of each batch calculated
    for m in range(M):
        # Intialize submodels
        Hmodel = NHNet()
        Hcriterion = nn.MSELoss()
        if torch.cuda.is_available():
            Hmodel = Hmodel.cuda()
            gamma_exp = gamma_exp.cuda()
            Hcriterion = Hcriterion.cuda()
            
        # Start Training
        epoch = 5
        Hmodel.train()
        Hoptimizer = optim.Adam(Hmodel.parameters(), lr=0.001)
        for i in range(epoch):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                # Create one-hot label target tensor
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                # Calculate F(x)
                Hoptimizer.zero_grad()
                output = initial_model(data)
                # Calculate the output from all the models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        output = output.cuda()
                        model = model.cuda()
                    output = output + gamma_exp[j] * model(data)
#                 print("output is:", output)
                #Convert into Onehot label so that it would be able to calculate the residual
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                #Calculate Residual
#                 print("target_onehot is:", target_onehot)
#                 print("output is:", output)
                residual = mseresidual(target_onehot, output)
                houtput = Hmodel(data)
#                 print("houtput is:", houtput)
                residual = residual + 1e-6
#                 print(residual)
                residual = residual.type(torch.cuda.FloatTensor)
                houtput = houtput.type(torch.cuda.FloatTensor)
                #Calculate the loss
                loss = Hcriterion(houtput, residual)
                loss.backward(retain_graph = True)
                Hoptimizer.step()
                # Print out current Process
#                 if (batch_idx + 1)% 100 == 0 and i % 10 == 0:
                print('Train Epoch: Model Number: {} {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        m+1,i, (batch_idx + 1) * len(data), len(train_loader.dataset),
                        100. * (batch_idx + 1) / len(train_loader), loss.item()))
        models.append(Hmodel)
        
        # Initialize a random gamma
        gamma = torch.rand(1, requires_grad=True, device="cuda")
        Goptimizer = optim.Adam([gamma], lr=0.01)
        Gcriterion = nn.MSELoss()
        # Start finding the best gamma
        for i in range(20):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                    Hmodel = Hmodel.cuda()
                    gamma = gamma.cuda()
                    
                #Calculate the initial output
                Goptimizer.zero_grad()  
                output = initial_model(data)
                #Calculate the final output by combining all previous models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        model = model.cuda()
                        output = output.cuda()
                        gamma_temp = gamma_exp[j]
                        gamma_temp = gamma_temp.cuda()
                    output = output + gamma_temp * model(data)
                # Covert into one-hot label
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                # Get the currect model output
                temp = Hmodel(data)
                # Find the current ensemble model output
                predicted = output + gamma * temp
                # Calculate the loss
                loss = Gcriterion(predicted, target_onehot)
                loss.backward(retain_graph = True)
                # Optimize the gamma
                Goptimizer.step()  
                
        gamma_exp[m] = gamma
    print(gamma_exp)
    return models, gamma_exp

## Start Gradient Boosting Training

In [29]:
num_of_models = 3
models, gamma_exp = GradientBoosting(initial_model, num_of_models)

Train Epoch: Model Number: 1 0 [64/60000 (0%)]	Loss: 0.025850
Train Epoch: Model Number: 1 0 [128/60000 (0%)]	Loss: 0.025335
Train Epoch: Model Number: 1 0 [192/60000 (0%)]	Loss: 0.032024
Train Epoch: Model Number: 1 0 [256/60000 (0%)]	Loss: 0.028735
Train Epoch: Model Number: 1 0 [320/60000 (1%)]	Loss: 0.028685
Train Epoch: Model Number: 1 0 [384/60000 (1%)]	Loss: 0.031690
Train Epoch: Model Number: 1 0 [448/60000 (1%)]	Loss: 0.022638
Train Epoch: Model Number: 1 0 [512/60000 (1%)]	Loss: 0.025709
Train Epoch: Model Number: 1 0 [576/60000 (1%)]	Loss: 0.034595
Train Epoch: Model Number: 1 0 [640/60000 (1%)]	Loss: 0.037884
Train Epoch: Model Number: 1 0 [704/60000 (1%)]	Loss: 0.037979
Train Epoch: Model Number: 1 0 [768/60000 (1%)]	Loss: 0.022472
Train Epoch: Model Number: 1 0 [832/60000 (1%)]	Loss: 0.043541
Train Epoch: Model Number: 1 0 [896/60000 (1%)]	Loss: 0.022473
Train Epoch: Model Number: 1 0 [960/60000 (2%)]	Loss: 0.019343
Train Epoch: Model Number: 1 0 [1024/60000 (2%)]	Loss: 0

Train Epoch: Model Number: 1 0 [8896/60000 (15%)]	Loss: 0.027167
Train Epoch: Model Number: 1 0 [8960/60000 (15%)]	Loss: 0.021946
Train Epoch: Model Number: 1 0 [9024/60000 (15%)]	Loss: 0.035248
Train Epoch: Model Number: 1 0 [9088/60000 (15%)]	Loss: 0.019203
Train Epoch: Model Number: 1 0 [9152/60000 (15%)]	Loss: 0.022749
Train Epoch: Model Number: 1 0 [9216/60000 (15%)]	Loss: 0.034306
Train Epoch: Model Number: 1 0 [9280/60000 (15%)]	Loss: 0.030286
Train Epoch: Model Number: 1 0 [9344/60000 (16%)]	Loss: 0.031445
Train Epoch: Model Number: 1 0 [9408/60000 (16%)]	Loss: 0.034786
Train Epoch: Model Number: 1 0 [9472/60000 (16%)]	Loss: 0.022425
Train Epoch: Model Number: 1 0 [9536/60000 (16%)]	Loss: 0.031485
Train Epoch: Model Number: 1 0 [9600/60000 (16%)]	Loss: 0.028997
Train Epoch: Model Number: 1 0 [9664/60000 (16%)]	Loss: 0.030927
Train Epoch: Model Number: 1 0 [9728/60000 (16%)]	Loss: 0.034236
Train Epoch: Model Number: 1 0 [9792/60000 (16%)]	Loss: 0.027504
Train Epoch: Model Number

Train Epoch: Model Number: 1 0 [17536/60000 (29%)]	Loss: 0.035712
Train Epoch: Model Number: 1 0 [17600/60000 (29%)]	Loss: 0.016223
Train Epoch: Model Number: 1 0 [17664/60000 (29%)]	Loss: 0.033416
Train Epoch: Model Number: 1 0 [17728/60000 (30%)]	Loss: 0.027716
Train Epoch: Model Number: 1 0 [17792/60000 (30%)]	Loss: 0.030998
Train Epoch: Model Number: 1 0 [17856/60000 (30%)]	Loss: 0.030768
Train Epoch: Model Number: 1 0 [17920/60000 (30%)]	Loss: 0.035700
Train Epoch: Model Number: 1 0 [17984/60000 (30%)]	Loss: 0.024756
Train Epoch: Model Number: 1 0 [18048/60000 (30%)]	Loss: 0.026548
Train Epoch: Model Number: 1 0 [18112/60000 (30%)]	Loss: 0.025427
Train Epoch: Model Number: 1 0 [18176/60000 (30%)]	Loss: 0.025284
Train Epoch: Model Number: 1 0 [18240/60000 (30%)]	Loss: 0.028589
Train Epoch: Model Number: 1 0 [18304/60000 (30%)]	Loss: 0.022120
Train Epoch: Model Number: 1 0 [18368/60000 (31%)]	Loss: 0.027885
Train Epoch: Model Number: 1 0 [18432/60000 (31%)]	Loss: 0.028045
Train Epoc

Train Epoch: Model Number: 1 0 [26176/60000 (44%)]	Loss: 0.019920
Train Epoch: Model Number: 1 0 [26240/60000 (44%)]	Loss: 0.017222
Train Epoch: Model Number: 1 0 [26304/60000 (44%)]	Loss: 0.028104
Train Epoch: Model Number: 1 0 [26368/60000 (44%)]	Loss: 0.022406
Train Epoch: Model Number: 1 0 [26432/60000 (44%)]	Loss: 0.021373
Train Epoch: Model Number: 1 0 [26496/60000 (44%)]	Loss: 0.020041
Train Epoch: Model Number: 1 0 [26560/60000 (44%)]	Loss: 0.026272
Train Epoch: Model Number: 1 0 [26624/60000 (44%)]	Loss: 0.020063
Train Epoch: Model Number: 1 0 [26688/60000 (44%)]	Loss: 0.031845
Train Epoch: Model Number: 1 0 [26752/60000 (45%)]	Loss: 0.028830
Train Epoch: Model Number: 1 0 [26816/60000 (45%)]	Loss: 0.027717
Train Epoch: Model Number: 1 0 [26880/60000 (45%)]	Loss: 0.030496
Train Epoch: Model Number: 1 0 [26944/60000 (45%)]	Loss: 0.025454
Train Epoch: Model Number: 1 0 [27008/60000 (45%)]	Loss: 0.030829
Train Epoch: Model Number: 1 0 [27072/60000 (45%)]	Loss: 0.022040
Train Epoc

Train Epoch: Model Number: 1 0 [34240/60000 (57%)]	Loss: 0.042182
Train Epoch: Model Number: 1 0 [34304/60000 (57%)]	Loss: 0.024584
Train Epoch: Model Number: 1 0 [34368/60000 (57%)]	Loss: 0.027407
Train Epoch: Model Number: 1 0 [34432/60000 (57%)]	Loss: 0.037884
Train Epoch: Model Number: 1 0 [34496/60000 (57%)]	Loss: 0.036981
Train Epoch: Model Number: 1 0 [34560/60000 (58%)]	Loss: 0.023339
Train Epoch: Model Number: 1 0 [34624/60000 (58%)]	Loss: 0.030449
Train Epoch: Model Number: 1 0 [34688/60000 (58%)]	Loss: 0.024678
Train Epoch: Model Number: 1 0 [34752/60000 (58%)]	Loss: 0.037493
Train Epoch: Model Number: 1 0 [34816/60000 (58%)]	Loss: 0.036927
Train Epoch: Model Number: 1 0 [34880/60000 (58%)]	Loss: 0.029090
Train Epoch: Model Number: 1 0 [34944/60000 (58%)]	Loss: 0.033833
Train Epoch: Model Number: 1 0 [35008/60000 (58%)]	Loss: 0.029015
Train Epoch: Model Number: 1 0 [35072/60000 (58%)]	Loss: 0.017887
Train Epoch: Model Number: 1 0 [35136/60000 (59%)]	Loss: 0.021189
Train Epoc

...

Train Epoch: Model Number: 1 4 [28992/60000 (48%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29056/60000 (48%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29120/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29184/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29248/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29312/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29376/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29440/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29504/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29568/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29632/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29696/60000 (49%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29760/60000 (50%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29824/60000 (50%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29888/60000 (50%)]	Loss: nan
Train Epoch: Model Number: 1 4 [29952/60000 (50%)]	Loss: nan
Train Epoch: Model Numbe

Train Epoch: Model Number: 1 4 [38464/60000 (64%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38528/60000 (64%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38592/60000 (64%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38656/60000 (64%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38720/60000 (64%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38784/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38848/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38912/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [38976/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39040/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39104/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39168/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39232/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39296/60000 (65%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39360/60000 (66%)]	Loss: nan
Train Epoch: Model Number: 1 4 [39424/60000 (66%)]	Loss: nan
Train Epoch: Model Numbe

Train Epoch: Model Number: 1 4 [47424/60000 (79%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47488/60000 (79%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47552/60000 (79%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47616/60000 (79%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47680/60000 (79%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47744/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47808/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47872/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [47936/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48000/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48064/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48128/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48192/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48256/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48320/60000 (80%)]	Loss: nan
Train Epoch: Model Number: 1 4 [48384/60000 (81%)]	Loss: nan
Train Epoch: Model Numbe

Train Epoch: Model Number: 1 4 [56448/60000 (94%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56512/60000 (94%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56576/60000 (94%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56640/60000 (94%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56704/60000 (94%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56768/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56832/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56896/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [56960/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57024/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57088/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57152/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57216/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57280/60000 (95%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57344/60000 (96%)]	Loss: nan
Train Epoch: Model Number: 1 4 [57408/60000 (96%)]	Loss: nan
Train Epoch: Model Numbe

RuntimeError: select(): index 0 out of range for tensor of size [0, 2] at dimension 0
Exception raised from select at ..\aten\src\ATen\native\TensorShape.cpp:889 (most recent call first):
00007FFF3D2275A200007FFF3D227540 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFF3D20444500007FFF3D2043D0 c10.dll!c10::IndexError::IndexError [<unknown file> @ <unknown line number>]
00007FFEE7EB3E5E00007FFEE7EB3870 torch_cpu.dll!at::native::select [<unknown file> @ <unknown line number>]
00007FFEE81A9A7E00007FFEE811D060 torch_cpu.dll!at::zeros_out [<unknown file> @ <unknown line number>]
00007FFEE7C50BFF00007FFEE7C46470 torch_cpu.dll!torch::nn::functional::BatchNormFuncOptions::~BatchNormFuncOptions [<unknown file> @ <unknown line number>]
00007FFEE80C115E00007FFEE80B8FA0 torch_cpu.dll!at::bucketize_out [<unknown file> @ <unknown line number>]
00007FFEE810976400007FFEE8109700 torch_cpu.dll!at::select [<unknown file> @ <unknown line number>]
00007FFEE937B37300007FFEE935E010 

## Save or Load submodels and optimized gamma

In [32]:
# Save all the models trained
for i in range(num_of_models):
    model = models[i]
    torch.save(model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(i) + '.pth')

In [33]:
# Save the optimized gamma
torch.save(gamma_exp, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')

In [ ]:
# Load the models from the local files
num_of_models = 3
models = []
for x in range(num_of_models):
    globals()['model%s' % x] = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(1) + '.pth')
    models.append(globals()['model%s' % x])
print(models)

In [ ]:
# Load the optimized gamma from the local files
gamma_exp = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')
print(gamma_exp)

## Ensemble Model Accuracy

In [34]:
#Find the accuracy of the ensemble model
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

loss = 0
correct = 0
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = initial_model(data)
        for i in range(num_of_models):
            model = models[i]
            if torch.cuda.is_available():
                model = model.cuda()
                output = output.cuda()
                gamma_temp = gamma_exp[i]
                gamma_temp = gamma_temp.cuda()
            output = output + gamma_temp * model(data)
        loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
loss /= len(train_loader.dataset)  
print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.



Average loss: 1.5499, Accuracy: 54379/60000 (90.632%)



## CW Attack on Intial Model, Testing Robust Accuracy

In [35]:
# Attack the initial model using CW attack
initial_model.eval()

correct = 0
total = 0

cw_attack = CW(initial_model, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 69.22 %


## Creating Ensemble CW Attack

In [36]:
# Create ensemble CW attack
class Attack_ensemble(object):
    r"""
    Base class for all attacks.
    .. note::
        It automatically set device to the device where given model is.
        It temporarily changes the original model's training mode to `test`
        by `.eval()` only during an attack process.
    """
    def __init__(self, name, model, models, gamma):
        r"""
        Initializes internal attack state.
        Arguments:
            name (str) : name of an attack.
            model (torch.nn.Module): model to attack.
        """

        self.attack = name
        self.model = model
        self.models = models
        self.gamma = gamma
        self.model_name = str(model).split("(")[0]

        self.training = model.training
        self.device = next(model.parameters()).device
        
        self._targeted = 1
        self._attack_mode = 'original'
        self._return_type = 'float'

    def forward(self, *input):
        r"""
        It defines the computation performed at every call.
        Should be overridden by all subclasses.
        """
        raise NotImplementedError
        
    def set_attack_mode(self, mode):
        r"""
        Set the attack mode.
  
        Arguments:
            mode (str) : 'original' (DEFAULT)
                         'targeted' - Use input labels as targeted labels.
                         'least_likely' - Use least likely labels as targeted labels.
        """
        if self._attack_mode is 'only_original':
            raise ValueError("Changing attack mode is not supported in this attack method.")
            
        if mode=="original":
            self._attack_mode = "original"
            self._targeted = 1
            self._transform_label = self._get_label
        elif mode=="targeted":
            self._attack_mode = "targeted"
            self._targeted = -1
            self._transform_label = self._get_label
        elif mode=="least_likely":
            self._attack_mode = "least_likely"
            self._targeted = -1
            self._transform_label = self._get_least_likely_label
        else:
            raise ValueError(mode + " is not a valid mode. [Options : original, targeted, least_likely]")
            
    def set_return_type(self, type):
        r"""
        Set the return type of adversarial images: `int` or `float`.
        Arguments:
            type (str) : 'float' or 'int'. (DEFAULT : 'float')
        """
        if type == 'float':
            self._return_type = 'float'
        elif type == 'int':
            self._return_type = 'int'
        else:
            raise ValueError(type + " is not a valid type. [Options : float, int]")

    def save(self, save_path, data_loader, verbose=True):
        r"""
        Save adversarial images as torch.tensor from given torch.utils.data.DataLoader.
        Arguments:
            save_path (str) : save_path.
            data_loader (torch.utils.data.DataLoader) : data loader.
            verbose (bool) : True for displaying detailed information. (DEFAULT : True)
        """
        self.model.eval()

        image_list = []
        label_list = []

        correct = 0
        total = 0

        total_batch = len(data_loader)

        for step, (images, labels) in enumerate(data_loader):
            adv_images = self.__call__(images, labels)

            image_list.append(adv_images.cpu())
            label_list.append(labels.cpu())

            if self._return_type == 'int':
                adv_images = adv_images.float()/255

            if verbose:
                outputs = self.model(adv_images)
                for i in range(len(self.models)):
                    sub_model = self.models[i]
                    outputs = outputs + self.gamma[i] * sub_model(adv_images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(self.device)).sum()

                acc = 100 * float(correct) / total
                print('- Save Progress : %2.2f %% / Accuracy : %2.2f %%' % ((step+1)/total_batch*100, acc), end='\r')

        x = torch.cat(image_list, 0)
        y = torch.cat(label_list, 0)
        torch.save((x, y), save_path)
        print('\n- Save Complete!')

        self._switch_model()
        
    def _transform_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        """
        return labels
        
    def _get_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return input labels.
        """
        return labels
    
    def _get_least_likely_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return least likely labels.
        """
        outputs = self.model(images)
        for i in range(len(self.models)):
            sub_model = self.models[i]
            outputs = outputs + self.gamma[i] * sub_model(images)
        _, labels = torch.min(outputs.data, 1)
        labels = labels.detach_()
        return labels
    
    def _to_uint(self, images):
        r"""
        Function for changing the return type.
        Return images as int.
        """
        return (images*255).type(torch.uint8)

    def _switch_model(self):
        r"""
        Function for changing the training mode of the model.
        """
        if self.training:
            self.model.train()
            for i in range(len(self.models)):
                self.models[i].train()
        else:
            self.model.eval()
            for i in range(len(self.models)):
                self.models[i].eval()

    def __str__(self):
        info = self.__dict__.copy()
        
        del_keys = ['model', 'attack']
        
        for key in info.keys():
            if key[0] == "_" :
                del_keys.append(key)
                
        for key in del_keys:
            del info[key]
        
        info['attack_mode'] = self._attack_mode
        if info['attack_mode'] == 'only_original' :
            info['attack_mode'] = 'original'
            
        info['return_type'] = self._return_type
        
        return self.attack + "(" + ', '.join('{}={}'.format(key, val) for key, val in info.items()) + ")"

    def __call__(self, *input, **kwargs):
        self.model.eval()
        for i in range(len(self.models)):
            self.models[i].eval()
        images = self.forward(*input, **kwargs)
        self._switch_model()

        if self._return_type == 'int':
            images = self._to_uint(images)

        return images

In [37]:
# Create Ensemble CW Attack
import warnings

class CW_Ensemble(Attack_ensemble):
    r"""
    CW in the paper 'Towards Evaluating the Robustness of Neural Networks'
    [https://arxiv.org/abs/1608.04644]
    Distance Measure : L2
        
    Arguments:
        model (nn.Module): model to attack.
        c (float): c in the paper. parameter for box-constraint. (DEFALUT : 1e-4)    
            :math:`minimize \Vert\frac{1}{2}(tanh(w)+1)-x\Vert^2_2+c\cdot f(\frac{1}{2}(tanh(w)+1))`    
        kappa (float): kappa (also written as 'confidence') in the paper. (DEFALUT : 0)
            :math:`f(x')=max(max\{Z(x')_i:i\neq t\} -Z(x')_t, - \kappa)`
        steps (int): number of steps. (DEFALUT : 1000)
        lr (float): learning rate of the Adam optimizer. (DEFALUT : 0.01)
        
    .. warning:: With default c, you can't easily get adversarial images. Set higher c like 1.
    
    Shape:
        - images: :math:`(N, C, H, W)` where `N = number of batches`, `C = number of channels`,        `H = height` and `W = width`. It must have a range [0, 1].
        - labels: :math:`(N)` where each value :math:`y_i` is :math:`0 \leq y_i \leq` `number of labels`.
        - output: :math:`(N, C, H, W)`.
          
    Examples::
        >>> attack = torchattacks.CW(model, targeted=False, c=1e-4, kappa=0, steps=1000, lr=0.01)
        >>> adv_images = attack(images, labels)
        
    .. note:: NOT IMPLEMENTED methods in the paper due to time consuming.
    
        (1) Binary search for c.
        
        (2) Choosing best L2 adversaries.
    """
    def __init__(self, model, models, gamma, c=1e-4, kappa=0, steps=1000, lr=0.01):
        super(CW_Ensemble, self).__init__("CW", model, models, gamma)
        self.c = c
        self.kappa = kappa
        self.steps = steps
        self.lr = lr

    def forward(self, images, labels):
        r"""
        Overridden.
        """
        images = images.to(self.device)
        labels = labels.to(self.device)
        labels = self._transform_label(images, labels)

        # f-function in the paper
        def f(x):
            outputs = self.model(x)
            for i in range(len(self.models)):
                sub_model = self.models[i]
                outputs = outputs + self.gamma[i] * sub_model(x)
            one_hot_labels = torch.eye(len(outputs[0]))[labels].to(self.device)

            i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
            j = torch.masked_select(outputs, one_hot_labels.bool())

            return torch.clamp(self._targeted*(j-i), min=-self.kappa)

        w = torch.zeros_like(images).to(self.device)
        w.detach_()
        w.requires_grad = True

        optimizer = optim.Adam([w], lr=self.lr)
        prev = 1e10

        for step in range(self.steps):

            a = 1/2*(nn.Tanh()(w) + 1)

            loss1 = nn.MSELoss(reduction='sum')(a, images)
            loss2 = torch.sum(self.c*f(a))

            cost = loss1 + loss2

            optimizer.zero_grad()
            cost.backward(retain_graph=True)
            optimizer.step()

            # Early Stop when loss does not converge.
            if step % (self.steps//10) == 0:
                if cost > prev:
                    warnings.warn("Early stopped because the loss is not converged.")
                    return (1/2*(nn.Tanh()(w) + 1)).detach()
                prev = cost

            # print('- CW Attack Progress : %2.2f %%        ' %((step+1)/self.steps*100), end='\r')

        adv_images = (1/2*(nn.Tanh()(w) + 1)).detach()

        return adv_images

## CW Attack Ensemble Model, Testing Robust Accuracy

In [38]:
#Attack the ensemble model
initial_model.eval()
for i in range(num_of_models):
    models[i].eval()

correct = 0
total = 0

cw_attack = CW_Ensemble(model = initial_model,models = models,gamma = gamma_exp, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        for i in range(num_of_models):
            sub_model = models[i]
            outputs = outputs + gamma_exp[i] * sub_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 70.03 %
